<a href="https://colab.research.google.com/github/christianasb/Python-Capstone-DataCamp-Mini-Projects/blob/main/crime-analysis-LA/Analyzing_Crime_in_LA_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Los Angeles skyline](la_skyline.jpg)

Los Angeles, California 😎. The City of Angels. Tinseltown. The Entertainment Capital of the World!

Known for its warm weather, palm trees, sprawling coastline, and Hollywood, along with producing some of the most iconic films and songs. However, as with any highly populated city, it isn't always glamorous and there can be a large volume of crime. That's where you can help!

You have been asked to support the Los Angeles Police Department (LAPD) by analyzing crime data to identify patterns in criminal behavior. They plan to use your insights to allocate resources effectively to tackle various crimes in different areas.

## The Data

They have provided you with a single dataset to use. A summary and preview are provided below.

It is a modified version of the original data, which is publicly available from Los Angeles Open Data.

# crimes.csv

| Column     | Description              |
|------------|--------------------------|
| `'DR_NO'` | Division of Records Number: Official file number made up of a 2-digit year, area ID, and 5 digits. |
| `'Date Rptd'` | Date reported - MM/DD/YYYY. |
| `'DATE OCC'` | Date of occurrence - MM/DD/YYYY. |
| `'TIME OCC'` | In 24-hour military time. |
| `'AREA NAME'` | The 21 Geographic Areas or Patrol Divisions are also given a name designation that references a landmark or the surrounding community that it is responsible for. For example, the 77th Street Division is located at the intersection of South Broadway and 77th Street, serving neighborhoods in South Los Angeles. |
| `'Crm Cd Desc'` | Indicates the crime committed. |
| `'Vict Age'` | Victim's age in years. |
| `'Vict Sex'` | Victim's sex: `F`: Female, `M`: Male, `X`: Unknown. |
| `'Vict Descent'` | Victim's descent:<ul><li>`A` - Other Asian</li><li>`B` - Black</li><li>`C` - Chinese</li><li>`D` - Cambodian</li><li>`F` - Filipino</li><li>`G` - Guamanian</li><li>`H` - Hispanic/Latin/Mexican</li><li>`I` - American Indian/Alaskan Native</li><li>`J` - Japanese</li><li>`K` - Korean</li><li>`L` - Laotian</li><li>`O` - Other</li><li>`P` - Pacific Islander</li><li>`S` - Samoan</li><li>`U` - Hawaiian</li><li>`V` - Vietnamese</li><li>`W` - White</li><li>`X` - Unknown</li><li>`Z` - Asian Indian</li> |
| `'Weapon Desc'` | Description of the weapon used (if applicable). |
| `'Status Desc'` | Crime status. |
| `'LOCATION'` | Street address of the crime. |

## Importing Data

In [3]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from io import StringIO

#read in the data from a Google Drive shareable link
orig_url="https://drive.google.com/file/d/1-Csx8-dGrY0Xs6CmNnbVDI01AO4h5T5x/view?usp=sharing"

file_id = orig_url.split('/')[-2] #extract file ID from url
dwn_url = 'https://drive.google.com/uc?export=download&id=' + file_id

#download and convert CSV content into a readable format
url = requests.get(dwn_url).text
csv_raw = StringIO(url)

#read the data into a pandas DataFrame
crimes = pd.read_csv(csv_raw, dtype={"TIME OCC": str})
crimes.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION
0,220314085,2022-07-22,2020-05-12,1110,Southwest,THEFT OF IDENTITY,27,F,B,NaN,Invest Cont,2500 S SYCAMORE AV
1,222013040,2022-08-06,2020-06-04,1620,Olympic,THEFT OF IDENTITY,60,M,H,NaN,Invest Cont,3300 SAN MARINO ST
2,220614831,2022-08-18,2020-08-17,1200,Hollywood,THEFT OF IDENTITY,28,M,H,NaN,Invest Cont,1900 TRANSIENT
3,231207725,2023-02-27,2020-01-27,0635,77th Street,THEFT OF IDENTITY,37,M,H,NaN,Invest Cont,6200 4TH AV
4,220213256,2022-07-14,2020-07-14,0900,Rampart,THEFT OF IDENTITY,79,M,B,NaN,Invest Cont,1200 W 7TH ST


In [4]:
# Check data types
crimes.dtypes

,0
DR_NO,int64
Date Rptd,object
DATE OCC,object
TIME OCC,object
AREA NAME,object
Crm Cd Desc,object
Vict Age,int64
Vict Sex,object
Vict Descent,object
Weapon Desc,object


## Finding the frequencies of crimes by the hour of occurence

In [5]:
#convert time to integer data type
crimes["TIME OCC"] = crimes["TIME OCC"].astype(int)

#extract hour from time by dividing by 100
crimes["Hour"] = crimes["TIME OCC"] // 100

#convert hour to integer data type
crimes["Hour"] = crimes["Hour"].astype(int)

#count occurences of each hour
crime_counts = crimes["Hour"].value_counts()

#highest frequency of crimes
peak_crime_hour = crime_counts.idxmax()
print(peak_crime_hour)


12


## Identifying the area with the most night crime

In [6]:
#filter the data for the relevant hours (10pm - 3:59am)
night_crimes = crimes[(crimes["Hour"] >= 22) | (crimes["Hour"] < 4)]

#count the number of crimes by area
night_crime_counts = night_crimes["AREA NAME"].value_counts()

#find area with highest frequency of night crimes
peak_night_crime_location = night_crime_counts.idxmax()
print(peak_night_crime_location)

Central


## Crimes by age group

In [7]:
#bin and label victim age into provided groups
age_bins = [0,17,25,34,44,54,64,float("inf")]
age_labels = ["0-17", "18-25", "26-34", "35-44", "45-54", "55-64", "65+"]

#bin Vict Age into groups
crimes["Age Group"] = pd.cut(crimes["Vict Age"], bins=age_bins, labels=age_labels, right=True)

#count number of crimes per age group
victim_ages = crimes["Age Group"].value_counts().reindex(age_labels, fill_value=0)
print(victim_ages)

Age Group
0-17      4528
18-25    28291
26-34    47470
35-44    42157
45-54    28353
55-64    20169
65+      14747
Name: count, dtype: int64


## 🧠 Interpretation & Recommendations for LAPD Resource Allocation

Based on the analysis of time, location, and victim demographics in this crime dataset, we suggest the following strategic actions:

### 🔍 Key Findings:
- **Peak crime hour:** 12:00 PM (noon), suggesting more incidents occur during midday than at night.
- **Night crime hotspot:** Central Division experiences the most crime between 10:00 PM and 3:59 AM, requiring enhanced late-night patrol in that area.
- **Most-affected age group:** Victims aged 26–34 report the highest number of incidents, followed by ages 35–44 and 18–25.

### 🚔 Recommended Actions:
1. **Increase police visibility in Central Division** from 10 PM–4 AM to deter or respond quickly to nighttime crimes.
2. **Adjust shift scheduling** to maintain high officer presence during the **midday peak** (11 AM–1 PM).
3. Launch **community outreach or crime prevention initiatives** targeting adults in the 26–44 age range, especially in high-crime neighborhoods.
4. Consider running a follow-up study comparing crime types and weapon use by time of day to further refine patrol strategies.

---